# Download NWM SR Forecast Data for July 4 - 7, 2025

**Author(s):** 

<ul style="line-height:1.5;">
<li>Nana Oye Djan <a href="mailto:ndjan@andrew.cmu.edu">(ndjan@andrew.cmu.edu)</a></li>
</ul>

**Last Updated:** 
17th July 2025

**Purpose:**

This notebook provides code to retrieve NOAA National Water Model Short Range data from Amazon Web Services (AWS) in netcdf format for Travis County, Texas for the July 4 - 7 catastrophic flood events.

**Description:**

This notebook downloads an ensmeble of 10 forecasts for each forecast time, from a predefined time to 10 hours ahead, run at different initialization times. For the July 4 - 7 events, the max discharge occured between July 5 and 6. To capture this peak, our predefined time was 5 PM UTC.  It takes a csv file of COMIDs, a layer in a geodatase with information on the geometries of the reaches (COMIDs) then retrieves data from AWS for Travis County, merges with the geodatabase layer, and saves them as shapefiles.

**Data Description:**

This notebook uses data developed and published by NOAA on Amazon Web Services (AWS) as described in detail in <a href="https://registry.opendata.aws/noaa-nwm-pds/">(this registry)</a></li> of open data entry. The National Water Model (NWM) is a water resources model that simulates and forecasts water budget variables, including snowpack, evapotranspiration, soil moisture and streamflow, over the entire continental United States (CONUS). It is operated by NOAA’s Office of Water Prediction. This bucket contains a four-week rollover of the Short Range Forecast model output and the corresponding forcing data for the model. The model is forced with meteorological data from the High Resolution Rapid Refresh (HRRR) and the Rapid Refresh (RAP) models. The Short Range Forecast configuration cycles hourly and produces hourly deterministic forecasts of streamflow and hydrologic states out to 18 hours.  It also uses information on Travis County flowlines provided <a href="http://www.hydroshare.org/resource/c95e654312204ce0b4d8e31e71cd4354">(here)</a></li>

**Software Requirements:**

This notebook uses Python v3.10.14 and requires the following specific Python libraries: 

> xarray: 2025.4.0     
   geopandas: 0.14.4  
   pandas: 2.3.0 
   s3fs: 2025.5.1  
   fsspec: 2025.5.1 \
   os: Python 3.10.14 (stdlib) \
   datetime / timedelta: Python 3.10.14 (stdlib) \
   re: Python 3.10.14 (stdlib)   

**Disclosure**
The code contained in this notebook was partially created and revised by ChatGPT, an AI language model developed by OpenAI

### 1. Setup

In [ ]:
#Import necessary packages
from datetime import datetime, timedelta, timezone
import pandas as pd
import geopandas as gpd
import xarray as xr
import s3fs
import fsspec
import re, os

### 2. Define important parameters and import necessary data

In [ ]:
# (change only these two lines if you pick another day/hour)
target_date          = datetime(2025, 7, 5).date()  # yyyy-mm-dd
latest_run_hr_fixed  = 17                           # “now” cycle (UTC)

#Import necessary files/Define necessary parameters
comid_csv      = "/Users/nanaoye/Documents/ArcGIS/Projects/Theme4DataRevised/Travis_Feature_IDs.csv"
flowlines_gpkg = "/Users/nanaoye/Documents/ArcGIS/Projects/Theme4DataRevised/Theme4Data.gdb"
flowline_layer = "P2FFlowlines"

comids        = pd.read_csv(comid_csv, dtype={"IDs": str})["IDs"].tolist()
flows         = gpd.read_file(flowlines_gpkg, layer=flowline_layer)
flows         = flows.rename(columns={"IDs":"feature_id"})
flows["feature_id"] = flows["feature_id"].astype(str)
comids_int    = [int(c) for c in comids]

print(f"Loaded {len(flows)} flowlines matching COMIDs.")

out_dir = "/Users/nanaoye/Library/CloudStorage/Box-Box/My Research/CUAHSI/SI_2025/sr_nwm_forecasts"
os.makedirs(out_dir, exist_ok=True)

#S3 bucket info
s3_bucket     = "noaa-nwm-pds"
forecast_path = "short_range"
variable      = "streamflow"
filename      = "channel_rt"
max_lead_hours= 10

Loaded 910 flowlines matching COMIDs.
Found cycles [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]Z on 2025-07-05
✔ Using t17Z as latest run


### 3. Define necessary functions

In [ ]:
# URL builder
def construct_s3_url(cycle_dt: datetime, lead_hr: int) -> str:
    date_str = cycle_dt.strftime("%Y%m%d")
    cycle_hr = f"{cycle_dt.hour:02d}"
    lead_str = f"{lead_hr:03d}"
    fname = f"nwm.t{cycle_hr}z.short_range.{filename}.f{lead_str}.conus.nc"
    return f"s3://{s3_bucket}/nwm.{date_str}/{forecast_path}/{fname}"

#Define function to calculate worst case discharge and most likely discharge
def collapse_with_runs(df):
    df = df.copy().rename(columns={"forecast_r":"forecast_run",
                                   "forecast_t":"forecast_time"})
    df["forecast_r_dt"] = pd.to_datetime(df["forecast_run"])
    df["forecast_t_dt"] = pd.to_datetime(df["forecast_time"])
    out = []
    for fid, grp in df.groupby("feature_id", sort=False):
        valid = grp[grp["forecast_r_dt"] <= grp["forecast_t_dt"]]
        valid["lead_sec"] = (valid["forecast_t_dt"]-valid["forecast_r_dt"]).dt.total_seconds()
        top2 = valid.nlargest(2, "lead_sec")
        if not top2.empty:
            ml_row = top2.loc[top2["streamflow"].idxmax()]
            ml_dis, ml_r, ml_t = ml_row["streamflow"], ml_row["forecast_run"], ml_row["forecast_time"]
        else:
            ml_dis = ml_r = ml_t = None
        wc_row = grp.loc[grp["streamflow"].idxmax()]
        out.append({"feature_id":fid,
                    "ml_dis":ml_dis,"ml_run":ml_r,"ml_time":ml_t,
                    "wc_dis":wc_row["streamflow"],
                    "wc_run":wc_row["forecast_run"],
                    "wc_time":wc_row["forecast_time"]})
    return pd.DataFrame(out)

# Function to turn any datetime64 columns into text (Shapefile-safe)
def scrub_datetimes(df, fmt="%Y-%m-%d %H:%M:%S"):
    for c in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[c]):
            df[c] = df[c].dt.strftime(fmt)
    return df


### 4. Download SR Forecast for target date: July 5

In [ ]:
# Initialize file system                                
fs      = s3fs.S3FileSystem(anon=True, client_kwargs={"region_name":"us-east-1"})
prefix  = f"{s3_bucket}/nwm.{target_date:%Y%m%d}/short_range/"
channel_files = [f for f in fs.find(prefix)
                 if f.endswith(".conus.nc") and ".channel_rt." in f]

if not channel_files:
    raise RuntimeError("No channel_rt files for selected date")

published_runs = {int(re.search(r"nwm\.t(\d{2})z", os.path.basename(p)).group(1))
                  for p in channel_files}
print(f"Found cycles {sorted(published_runs)}Z on {target_date}")

run_lead_map = {}
for p in channel_files:
    m_run  = re.search(r"nwm\.t(\d{2})z", os.path.basename(p))
    m_lead = re.search(r"\.f(\d{3})\.",  os.path.basename(p))
    if m_run and m_lead:
        run_hr, lead_hr = int(m_run.group(1)), int(m_lead.group(1))
        if 1 <= lead_hr <= max_lead_hours:
            run_lead_map[(run_hr, lead_hr)] = p

prefix_dt      = datetime.combine(target_date, datetime.min.time())
latest_run_hr  = latest_run_hr_fixed if latest_run_hr_fixed in published_runs else max(published_runs)
latest_run_dt  = prefix_dt + timedelta(hours=latest_run_hr)

# Determine latest initialization (forecast run) time                   
records_current = []
for lead_hr in range(1, max_lead_hours+1):
    key = run_lead_map.get((latest_run_hr, lead_hr))
    if not key: continue
    with fs.open(key,"rb") as f:
        ds = xr.open_dataset(f, engine="h5netcdf")
        ids = ds["feature_id"].values.astype(int)
        present = [cid for cid in comids_int if cid in ids]
        if not present: continue
        da = ds[variable].sel(feature_id=present).load()
    df = da.to_dataframe().reset_index()
    df["feature_id"] = df["feature_id"].astype(str)
    df = df.set_index("feature_id").reindex(comids).reset_index()
    df["valid_time"]   = latest_run_dt + timedelta(hours=lead_hr)
    df["forecast_run"] = latest_run_dt
    df["lead_hour"]    = lead_hr
    records_current.append(df)

df_current = pd.concat(records_current, ignore_index=True)
print(f"✔ Using t{latest_run_hr:02d}Z as latest run")

# Obtain Full ensemble for every valid hour on July 5 - 6 
records_ensemble = []
valid_times = [prefix_dt + timedelta(hours=h) for h in range(24)]  # 00–23 UTC

for vt in valid_times:
    for lead_hr in range(1, max_lead_hours+1):
        run_dt = vt - timedelta(hours=lead_hr)
        key    = run_lead_map.get((run_dt.hour, lead_hr))
        if not key: continue
        with fs.open(key,"rb") as f:
            ds = xr.open_dataset(f, engine="h5netcdf")
            ids = ds["feature_id"].values.astype(int)
            pres= [cid for cid in comids_int if cid in ids]
            if not pres: continue
            da = ds[variable].sel(feature_id=pres).load()
        dfe = da.to_dataframe().reset_index()
        dfe["feature_id"]   = dfe["feature_id"].astype(str)
        dfe = dfe.set_index("feature_id").reindex(comids).reset_index()
        dfe["forecast_time"]= vt
        dfe["forecast_run"] = run_dt
        dfe["lead_hour"]    = lead_hr
        records_ensemble.append(dfe)

df_all = pd.concat(records_ensemble, ignore_index=True)
df_all = df_all.rename(columns={"forecast_time":"forecast_time"})
df_all['forecast_run']  = pd.to_datetime(df_all['forecast_run'])
df_all['forecast_time'] = pd.to_datetime(df_all['forecast_time'])

# optional: restrict forecast times to only times in target date.
df_all = df_all[df_all["forecast_time"].dt.date == target_date].reset_index(drop=True)


### 5. Download shapefiles

In [ ]:

#1-h
next_hour = latest_run_dt + timedelta(hours=1)
df1_multi = df_all.query("forecast_time == @next_hour and lead_hour >= 1")[[
                "feature_id","forecast_time","forecast_run","streamflow"]]
df1_multi["streamflow"] = df1_multi["streamflow"].fillna(0)
collapse_1h = collapse_with_runs(df1_multi.rename(columns={"forecast_time":"forecast_t",
                                                           "forecast_run":"forecast_r"}))
for df in (flows, collapse_1h):
    df["feature_id"] = pd.to_numeric(df["feature_id"], errors="coerce").astype(int)

gdf1_multi = (
    flows[["LakeID","HydroID","From_Node","To_Node",
             "NextDownID","feature_id","order_","areasqkm","geometry"]]
      .merge(collapse_1h,on="feature_id",how="left")
)
gdf1_multi = scrub_datetimes(gdf1_multi)
gdf1_multi.to_file(os.path.join(out_dir,"sr_nwm_tc_1hour.shp"))

# 2-h
target_time = latest_run_dt + timedelta(hours=2)
df2_multi = df_all.query("forecast_time == @target_time")[[
                "feature_id","forecast_time","forecast_run","streamflow"]]
df2_multi["streamflow"] = df2_multi["streamflow"].fillna(0)
collapse_2h = collapse_with_runs(df2_multi.rename(columns={"forecast_time":"forecast_t",
                                                           "forecast_run":"forecast_r"}))
# after you build collapse_2h but *before* the merge — add this line
collapse_2h["feature_id"] = pd.to_numeric(collapse_2h["feature_id"],
                                          errors="coerce").astype("int")
keep_cols = ["LakeID","HydroID","From_Node","To_Node",
             "NextDownID","feature_id","order_","areasqkm","geometry"]
gdf2_multi = flows[keep_cols].merge(collapse_2h,on="feature_id",how="left")
gdf2_multi = scrub_datetimes(gdf2_multi)
gdf2_multi.to_file(os.path.join(out_dir,"sr_nwm_tc_2hour.shp"))

# Max-10 h discharge (existing)                                 
latest_run = df_current["forecast_run"].max()
df10 = (df_all.groupby("feature_id")["streamflow"]
              .max()
              .reset_index()
              .rename(columns={"streamflow":"streamflow"}))
df10["datetime"] = latest_run.strftime("%Y-%m-%d %H:%M:%S")
# after you build collapse_2h but *before* the merge — add this line
df10["feature_id"] = pd.to_numeric(df10["feature_id"],
                                          errors="coerce").astype("int")
gdf10 = flows[keep_cols].merge(df10,on="feature_id",how="left")
gdf10["streamflow"] = gdf10["streamflow"].fillna(0)
gdf10      = scrub_datetimes(gdf10)
gdf10.to_file(os.path.join(out_dir,"sr_nwm_tc_max10hr.shp"))

# Optional: download daily peak (between 00 - 23 UTC) on target date                                   
peak_df = (df_all.groupby("feature_id")["streamflow"]
                  .max()
                  .reset_index()
                  .rename(columns={"streamflow":"peak_dis"}))
peak_df["date"] = target_date.strftime("%Y-%m-%d")
peak_df["feature_id"] = pd.to_numeric(peak_df["feature_id"],
                                          errors="coerce").astype("int")
gdf_peak = flows[keep_cols].merge(peak_df,on="feature_id",how="left")
gdf_peak["peak_dis"] = gdf_peak["peak_dis"].fillna(0)
gdf_peak   = scrub_datetimes(gdf_peak)
gdf_peak.to_file(os.path.join(out_dir,"sr_nwm_tc_20250704_peak.shp"))

print("✓ Finished: 1-h, 2-h, max-10 h, and daily-peak shapefiles written")

✓ Finished: 1-h, 2-h, max-10 h, and daily-peak shapefiles written


### 6. Create shapefiles for NWM Analysis Assimilation Data for July 4 - 7, 2025

In [ ]:
#Read in csv file comtaining peak streamflows for NWM Analysis Assimilation with data assimilation and no (USGS) data assimilation
da_no_da = pd.read_csv('/Users/nanaoye/Library/CloudStorage/Box-Box/My Research/CUAHSI/SI_2025/nwm_model_da_allq_120sec_20250704_nwm.csv')
da_no_da

,ID,q_cfs_da,q_cfs_no_da
0,5671171,9474.442607,19330.954902
1,5671165,9474.442607,19330.954902
2,5671187,4503.178951,14442.636764
3,5671185,5485.485901,13432.715701
4,5671181,6417.461499,11400.223133
...,...,...,...
911,1629525,138.300948,138.300948
912,1629809,138.356851,138.356851
913,1629805,23.226317,23.226317
914,1629803,23.218592,23.218592


In [ ]:
#Create shapefile for data assimilation only flows
da_only = da_no_da[['ID', 'q_cfs_da']].copy()
da_only.rename(columns = {'ID': 'feature_id'}, inplace=True)
da_only['streamflow'] = da_only['q_cfs_da']/35.314666212661
da_only
keep_cols = ["LakeID","HydroID","From_Node","To_Node",
             "NextDownID","feature_id","order_","areasqkm","geometry"]
da_gdf= flows[keep_cols].merge(da_only,on="feature_id",how="left")
da_gdf
da_gdf.to_file(os.path.join(out_dir,"jul4_da_max.shp"))

In [ ]:
#Create shapefile for no data assimilation only flows
no_da_only = da_no_da[['ID', 'q_cfs_no_da']].copy()
no_da_only.rename(columns = {'ID': 'feature_id'}, inplace=True)
no_da_only['streamflow'] = no_da_only['q_cfs_no_da']/35.314666212661
no_da_only
keep_cols = ["LakeID","HydroID","From_Node","To_Node",
             "NextDownID","feature_id","order_","areasqkm","geometry"]
no_da_gdf= flows[keep_cols].merge(no_da_only,on="feature_id",how="left")
no_da_gdf
no_da_gdf.to_file(os.path.join(out_dir,"jul4_no_da_max.shp"))

/var/folders/bq/24vf7y9j0x75b0vr13pfwvf40000gn/T/ipykernel_55671/871627407.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  no_da_gdf.to_file(os.path.join(out_dir,"jul4_no_da_max.shp"))
